In [2]:
import datetime

def f(v):
    try:
        dt = datetime.datetime.strptime(date_text, '%d/%d/%Y')
        dt.month()
    except:
        return None
    


In [3]:
f('12/06/2008')

In [8]:
dt = datetime.datetime.strptime('12/06/2008 08:45:00', '%d/%m/%Y')

ValueError: unconverted data remains:  08:45:00

In [7]:
dt.month

6

In [9]:
s = '12/06/2008 08:45:00'

In [10]:
s[3:4]

'0'